# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
# Import necessary libraries
import pandas as pd

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Display the first few rows of the dataset to get an overview
print(data.head())

# Check the basic statistics of the dataset
print(data.describe())

# Check the number of unique values in the 'id' column
unique_ids = data['id'].nunique()
print(f"Number of unique IDs: {unique_ids}")

# Check the number of viewers who clicked
viewers_clicked = data[data['action'] == 'click']['id'].nunique()
print(f"Number of viewers who clicked: {viewers_clicked}")

# Check for anomalies - anyone clicked without viewing?
anomalies = data[data['action'] == 'click']['id'].isin(data[data['action'] == 'view']['id']).sum()
print(f"Number of anomalies (clicked without viewing): {anomalies}")

# Check for overlap between control and experiment groups
control_group = data[data['group'] == 'control']['id']
experiment_group = data[data['group'] == 'experiment']['id']
overlap = control_group.isin(experiment_group).sum()
print(f"Number of overlapping users between control and experiment groups: {overlap}")

# Explore the distribution of actions in control and experiment groups
action_distribution = data.groupby(['group', 'action']).size().unstack()
print(action_distribution)


                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
                  id
count    8188.000000
mean   564699.749878
std    219085.845672
min    182988.000000
25%    373637.500000
50%    566840.500000
75%    758078.000000
max    937217.000000
Number of unique IDs: 6328
Number of viewers who clicked: 1860
Number of anomalies (clicked without viewing): 1860
Number of overlapping users between control and experiment groups: 0
action      click  view
group                  
control       932  3332
experiment    928  2996


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [3]:
#Your code here
from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(data['group'], data['action'])

# Print the contingency table
print(contingency_table)

# Conduct the chi-squared test
chi2, p, _, _ = chi2_contingency(contingency_table)

# Print the test statistic and p-value
print(f"Chi-squared test statistic: {chi2}")
print(f"P-value: {p}")

# Determine statistical significance
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis. There is a significant difference between the groups.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the groups.")


action      click  view
group                  
control       932  3332
experiment    928  2996
Chi-squared test statistic: 3.636160051233291
P-value: 0.056537191086915774
Fail to reject the null hypothesis. There is no significant difference between the groups.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [4]:
#Your code here
# Calculate click-through rate for the control group
control_click_rate = contingency_table.loc['control', 'click'] / contingency_table.loc['control'].sum()

# Apply the click-through rate to the number of viewers in the experiment group
expected_clicks_experiment = control_click_rate * contingency_table.loc['experiment'].sum()

# Print the expected number of clicks for the experiment group
print(f"Expected number of clicks for the experiment group: {expected_clicks_experiment}")


Expected number of clicks for the experiment group: 857.6848030018762


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [5]:
#Your code here
import numpy as np

# Calculate the standard deviation for the control group
n_control = contingency_table.loc['control'].sum()
p_control = control_click_rate
std_dev_control = np.sqrt(n_control * p_control * (1 - p_control))

# Calculate the actual number of clicks for the experiment group
actual_clicks_experiment = contingency_table.loc['experiment', 'click']

# Calculate the number of standard deviations
num_std_dev = (actual_clicks_experiment - expected_clicks_experiment) / std_dev_control

# Print the number of standard deviations
print(f"Number of standard deviations from the estimate: {num_std_dev}")


Number of standard deviations from the estimate: 2.6055375912248526


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [6]:
#Your code here
from scipy.stats import norm

# Calculate the p-value using the normal distribution
p_value = norm.cdf(num_std_dev)

# Print the p-value
print(f"P-value based on the normal distribution: {p_value}")


P-value based on the normal distribution: 0.9954134896397755


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [7]:
"""The result obtained from the calculation of the p-value using the normal distribution based on the z-score should generally align with the result from the previous chi-squared test."""

'The result obtained from the calculation of the p-value using the normal distribution based on the z-score should generally align with the result from the previous chi-squared test.'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.